In [1]:
library("DESeq2")
setwd("~/relmapping")

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.max, which.min

Warning message:


In [2]:
fp <- "~/relmapping/atac728/tg_se.bwa_se.rm_unmapped.rm_chrM.rm_blacklist.rm_q10.macs2_se_extsize150_shiftm75_keepdup_all_noSPMR.sfnorm/atac728_wt.tg_se.bwa_se.rm_unmapped.rm_chrM.rm_blacklist.rm_q10.macs2_se_extsize150_shiftm75_keepdup_all_noSPMR.sfnorm_counts.tsv"

In [3]:
countData <- read.table(fp, header=TRUE, sep="\t")#, row.names=1)

In [4]:
cond <- relevel(factor(c(
    "wt_emb", "wt_emb",
    "wt_l1", "wt_l1",
    "wt_l2", "wt_l2",
    "wt_l3", "wt_l3",
    "wt_l4", "wt_l4",
    "wt_ya", "wt_ya",
    "glp1_ya", "glp1_ya",
    "glp1_d3", "glp1_d3",
    "glp1_d7", "glp1_d7",
    "glp1_d10", "glp1_d10",
    "glp1_d14", "glp1_d14")), "wt_emb")

In [5]:
colData <- data.frame(row.names=colnames(countData), condition=cond)

In [6]:
dds <- DESeqDataSetFromMatrix(countData=countData, colData=colData, design=~condition)
dds <- estimateSizeFactors(dds)

In [7]:
sizeFactors(dds)

wt_emb_rep1   wt_emb_rep2    wt_l1_rep1    wt_l1_rep2    wt_l2_rep1 
    0.9794446     0.6799207     0.7216237     0.7899458     0.6037461 
   wt_l2_rep2    wt_l3_rep1    wt_l3_rep2    wt_l4_rep1    wt_l4_rep2 
    1.2324217     0.9012524     1.1241031     1.4025560     1.2696827 
   wt_ya_rep1    wt_ya_rep2  glp1_ya_rep1  glp1_ya_rep2  glp1_d3_rep1 
    0.8067816     1.2863469     1.0137182     1.4695410     0.9490625 
 glp1_d3_rep2  glp1_d7_rep1  glp1_d7_rep2 glp1_d10_rep1 glp1_d10_rep2 
    0.9525726     0.8912030     1.0732622     0.9673382     1.3719287 
glp1_d14_rep1 glp1_d14_rep2 
    0.8240752     0.9629528

In [8]:
estimateSizeFactorsForMatrix(countData)

wt_emb_rep1   wt_emb_rep2    wt_l1_rep1    wt_l1_rep2    wt_l2_rep1 
    0.9794446     0.6799207     0.7216237     0.7899458     0.6037461 
   wt_l2_rep2    wt_l3_rep1    wt_l3_rep2    wt_l4_rep1    wt_l4_rep2 
    1.2324217     0.9012524     1.1241031     1.4025560     1.2696827 
   wt_ya_rep1    wt_ya_rep2  glp1_ya_rep1  glp1_ya_rep2  glp1_d3_rep1 
    0.8067816     1.2863469     1.0137182     1.4695410     0.9490625 
 glp1_d3_rep2  glp1_d7_rep1  glp1_d7_rep2 glp1_d10_rep1 glp1_d10_rep2 
    0.9525726     0.8912030     1.0732622     0.9673382     1.3719287 
glp1_d14_rep1 glp1_d14_rep2 
    0.8240752     0.9629528

In [9]:
estimateSizeFactorsForMatrixMedianScaled <- function(counts, locfunc = stats::median, geoMeans, controlGenes) {
    if (missing(geoMeans)) {
        incomingGeoMeans <- FALSE
        loggeomeans <- rowMeans(log(counts))
        geomeansMedian <- exp(locfunc(loggeomeans[is.finite(loggeomeans)]))
        print(geomeansMedian)
    }
    #else {
    #    incomingGeoMeans <- TRUE
    #    if (length(geoMeans) != nrow(counts)) {
    #        stop("geoMeans should be as long as the number of rows of counts")
    #    }
    #    loggeomeans <- log(geoMeans)
    #}
    if (all(is.infinite(loggeomeans))) {
        stop("every gene contains at least one zero, cannot compute log geometric means")
    }
    sf <- if (missing(controlGenes)) {
        print("missing(controlGenes)")
        apply(counts, 2, function(cnts) {
            exp(locfunc((log(cnts) - loggeomeans)[is.finite(loggeomeans) & cnts > 0]))
        })
    }
    #else {
    #    print("not missing(controlGenes)")
    #    if (!(is.numeric(controlGenes) | is.logical(controlGenes))) {
    #        stop("controlGenes should be either a numeric or logical vector")
    #    }
    #    loggeomeansSub <- loggeomeans[controlGenes]
    #    apply(counts[controlGenes, , drop = FALSE], 2, function(cnts) {
    #        exp(locfunc((log(cnts) - loggeomeansSub)[is.finite(loggeomeansSub) & cnts > 0]))
    #    })
    #}
    #if (incomingGeoMeans) {
    #    print("incomingGeoMeans")
    #    sf <- sf/exp(mean(log(sf)))
    #}
    print(sf)
    print(1 / sf)
    print(1 / (sf * geomeansMedian))
}

estimateSizeFactorsForMatrixMedianScaled(countData)

[1] 97.75643
[1] "missing(controlGenes)"
  wt_emb_rep1   wt_emb_rep2    wt_l1_rep1    wt_l1_rep2    wt_l2_rep1 
    0.9794446     0.6799207     0.7216237     0.7899458     0.6037461 
   wt_l2_rep2    wt_l3_rep1    wt_l3_rep2    wt_l4_rep1    wt_l4_rep2 
    1.2324217     0.9012524     1.1241031     1.4025560     1.2696827 
   wt_ya_rep1    wt_ya_rep2  glp1_ya_rep1  glp1_ya_rep2  glp1_d3_rep1 
    0.8067816     1.2863469     1.0137182     1.4695410     0.9490625 
 glp1_d3_rep2  glp1_d7_rep1  glp1_d7_rep2 glp1_d10_rep1 glp1_d10_rep2 
    0.9525726     0.8912030     1.0732622     0.9673382     1.3719287 
glp1_d14_rep1 glp1_d14_rep2 
    0.8240752     0.9629528 
  wt_emb_rep1   wt_emb_rep2    wt_l1_rep1    wt_l1_rep2    wt_l2_rep1 
    1.0209868     1.4707598     1.3857637     1.2659096     1.6563254 
   wt_l2_rep2    wt_l3_rep1    wt_l3_rep2    wt_l4_rep1    wt_l4_rep2 
    0.8114106     1.1095671     0.8895981     0.7129840     0.7875984 
   wt_ya_rep1    wt_ya_rep2  glp1_ya_rep1  glp1_y

In [38]:
# sf <- raw DESeq sizefactors
sf <- sizeFactors(dds)
sf

wt_emb_rep1 wt_emb_rep2  wt_l1_rep1  wt_l1_rep2  wt_l2_rep1  wt_l2_rep2 
  1.0201809   0.7064357   0.7123560   0.7914698   0.6285727   1.2545553 
 wt_l3_rep1  wt_l3_rep2  wt_l4_rep1  wt_l4_rep2  wt_ya_rep1  wt_ya_rep2 
  0.9120338   1.1748503   1.5247222   1.3548893   0.8251487   1.4028264

In [40]:
# geomeansMedian <- median counts in pseudo-reference sample
loggeomeans <- rowMeans(log(countData))
geomeansMedian <- exp(stats::median(loggeomeans[is.finite(loggeomeans)]))
geomeansMedian

[1] 109.3102

In [42]:
# final ATAC-seq normalisation factors
1 / (sf * geomeansMedian)

wt_emb_rep1 wt_emb_rep2  wt_l1_rep1  wt_l1_rep2  wt_l2_rep1  wt_l2_rep2 
0.008967310 0.012949910 0.012842284 0.011558594 0.014554050 0.007292048 
 wt_l3_rep1  wt_l3_rep2  wt_l4_rep1  wt_l4_rep2  wt_ya_rep1  wt_ya_rep2 
0.010030635 0.007786761 0.005999964 0.006752049 0.011086823 0.006521319